# 1. MNIST 데이터셋으로 분류기를 만들어 테스트 세트에서 97% 정확도를 달성해보세요.

In [7]:
from sklearn.datasets import fetch_openml
from matplotlib import pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score

mnist = fetch_openml('mnist_784', version=1, as_frame=False)
mnist.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [8]:
train = mnist['data']
test = mnist['target']

In [10]:
# 학습, 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(train, test, train_size=0.8, random_state=42)

# **KNeighborsClassifier**  
n_neighbors: 분류 시 고려할 인접 샘플 수

wieights(default='uniform'): distance로 설정하면 가까울수록 큰 가중치 부여하여 계산

![image.png](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Ft1.daumcdn.net%2Fcfile%2Ftistory%2F9931AB3D5A634B8626)

# **GridSearch와 RandomizedSearch**  
최적의 하이퍼 파라미터를 찾기 위한 방법  

GridSearchCV: 파라미터별 후보 값들을 주고 가능한 모든 조합을 탐색  
RandomizedSearch: 파라미터별 후보 값들 혹은 값의 범위를 주고 n_iter 수만큼 random하게 조합하는 과정을 반복

In [ ]:
# KNeighborsClassifier로 학습
knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train, y_train)

# Randomized Search 사용해 학습
params = {
    'n_neighbors' : [3, 4, 5],
    'weights' : ['uniform', 'distance'],
    # 'algorithm' : ['auto', 'ball_tree', 'kd_tree']
}
rs_knn_clf = RandomizedSearchCV(
    param_distributions = params,
    estimator = knn_clf,
    n_iter = 10, # 10개의 조합만큼 random search
    cv = 5, # 분할 검증 횟수
    verbose = 3,
)
rs_knn_clf.fit(X_train, y_train)
print(rs_knn_clf.best_params_)
print(rs_knn_clf.best_score_)

# 예측
y_pred1 = knn_clf.predict(X_test)
y_pred2 = rs_knn_clf.predict(X_test)
acc1 = accuracy_score(y_test, y_pred1)
acc2 = accuracy_score(y_test, y_pred2)
print(acc1)
print(acc2)


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END ....n_neighbors=3, weights=uniform;, score=0.969 total time=  46.9s
[CV 2/5] END ....n_neighbors=3, weights=uniform;, score=0.969 total time=  39.0s
[CV 3/5] END ....n_neighbors=3, weights=uniform;, score=0.972 total time=  36.3s
[CV 4/5] END ....n_neighbors=3, weights=uniform;, score=0.971 total time=  37.4s
[CV 5/5] END ....n_neighbors=3, weights=uniform;, score=0.970 total time=  42.0s
[CV 1/5] END ...n_neighbors=3, weights=distance;, score=0.971 total time=  36.7s
[CV 2/5] END ...n_neighbors=3, weights=distance;, score=0.970 total time=  37.0s
[CV 3/5] END ...n_neighbors=3, weights=distance;, score=0.973 total time=  37.7s
[CV 4/5] END ...n_neighbors=3, weights=distance;, score=0.972 total time=  35.7s
[CV 5/5] END ...n_neighbors=3, weights=distance;, score=0.971 total time=  37.7s
[CV 1/5] END ....n_neighbors=4, weights=uniform;, score=0.967 total time=  44.8s
[CV 2/5] END ....n_neighbors=4, weights=uniform;,

# 2. 픽셀 이동을 통한 데이터 증강

# **numpy array와 list의 차이점**

list: [1, 2, 3, 4, 5]  
np.array: array([1, 2, 3, 4, 5])  

넘파이 배열은 **팬시 인덱싱** 가능  
팬시인덱싱: 정수나 boolean 값을 갖는 다른 넘파이 배열로 인덱싱을 할 수 있는 기능  
-> 배열로 인덱싱을 하기 위해서는 리스트를 넘파이 배열로 바꿔줘야됨.



# **넘파이 배열과 append 함수**  
**넘파이 배열은 크기가 고정**돼있어 배열 자체에 추가를 하는 함수를 사용할 수 없음.  

a = np.array([1, 2, 3])이라는 배열이 있을 때 a.append(4)는 배열 a에 직접적으로 변형을 가하기 때문에 사용 불가  
대신 new_a = np.append(a, 4)와 같이 새로운 배열에 추가해주는 방식으로 append를 사용 가능


In [11]:
X_train[0].shape

(784,)

In [12]:
from scipy.ndimage import shift
import numpy as np

def shift_image(image, dx, dy):
  # image는 X_train에서 받아온 데이터
  # X_train은 1차원 넘파이배열(ndarray), 이미지형태 조작은 2D 배열로 바꿔줘야됨.
  image = image.reshape((28, 28))
  shifted_image = shift(image, [dy, dx], cval=0)
  return shifted_image.reshape([-1])

X_train_augmented = list(X_train.copy()) # append 함수 사용 위해 lsit로 바꿈
y_train_augmented = list(y_train.copy())

for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
  for img, label in zip(X_train, y_train):
    X_train_augmented.append(shift_image(img, dx, dy))
    y_train_augmented.append(label)

# shift_image 함수에서 넘파이 배열로 되더라도 X_train_augmented에는 리스트 형태로 담김.
# 팬시 인덱싱 위해서는 형변환 필요
X_train_augmented = np.array(X_train_augmented)
y_train_augmented = np.array(y_train_augmented)

shuffle_idx = np.random.permutation(len(X_train_augmented))
X_train_augmented = X_train_augmented[shuffle_idx]
y_train_augmented = y_train_augmented[shuffle_idx]

# KNeighborsClassifier로 학습
knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train_augmented, y_train_augmented)

# Randomized Search 사용해 학습
params = {
    'n_neighbors' : range(3, 6),
    'weights' : ['uniform', 'distance'],
    # 'algorithm' : ['auto', 'ball_tree', 'kd_tree']
}
rs_knn_clf = RandomizedSearchCV(
    param_distributions = params,
    estimator = knn_clf,
    n_iter = 5,
    cv = 3,
    verbose = 1,
)
#rs_knn_clf.fit(X_train_augmented, y_train_augmented)
#print(rs_knn_clf.best_params_)
#print(rs_knn_clf.best_score_)

# 예측
y_pred1 = knn_clf.predict(X_test)
#y_pred2 = rs_knn_clf.predict(X_test)
acc1 = accuracy_score(y_test, y_pred1)
#acc2 = accuracy_score(y_test, y_pred2)
print(acc1)
#print(acc2)



0.9789285714285715
